# Machine Learning - Out-of-Sample Forecasts with ARIMA

In this notebook, we are going to explore a couple of diffrent machine learning models to predict time-series data.

Here is a link to all articles/tutorials:
 - [Time Series Archive](http://machinelearningmastery.com/category/time-series/)
 
Here are links to specific articles:
 - [How to Make Out-of-Sample Forecasts with ARIMA in Python](http://machinelearningmastery.com/make-sample-forecasts-arima-python/)
 - [Sensitivity Analysis of History Size to Forecast Skill with ARIMA in Python](http://machinelearningmastery.com/sensitivity-analysis-history-size-forecast-skill-arima-python/)
 - [Feature Selection for Time Series Forecasting with Python](http://machinelearningmastery.com/feature-selection-time-series-forecasting-python/)
 - [Simple Time Series Forecasting Models to Test So That You Don’t Fool Yourself](http://machinelearningmastery.com/simple-time-series-forecasting-models/)
 - [Autoregression Models for Time Series Forecasting With Python](http://machinelearningmastery.com/autoregression-models-time-series-forecasting-python/)

In [1]:
import pandas as pd
import numpy as np

import warnings

from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA

import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)

from plotly.graph_objs import *

## Out-of-Sample Forecasts with ARIMA

This machine learning technique is broken down into the following 5 steps:

1. Dataset Description
2. Split Dataset
3. Develop Model
4. One-Step Out-of-Sample Forecast
5. Multi-Step Out-of-Sample Forecast

### Dataset Description

In [2]:
# load dataset
data = pd.read_csv('data/slo_weather_history.csv', index_col=0)

# display first few rows
data.head()

,dew_point_f_avg,dew_point_f_high,dew_point_f_low,events,humidity_%_avg,humidity_%_high,humidity_%_low,precip_in_sum,sea_level_press_in_avg,sea_level_press_in_high,sea_level_press_in_low,temp_f_avg,temp_f_high,temp_f_low,visibility_mi_avg,visibility_mi_high,visibility_mi_low,wind_gust_mph_high,wind_mph_avg,wind_mph_high
date,,,,,,,,,,,,,,,,,,,,
2012-01-01,44.0,50.0,34.0,Fog,80.0,100.0,25.0,0.00,30.15,30.23,30.08,56.0,73.0,39.0,6.0,10.0,0.0,0.0,1.0,8.0
2012-01-02,47.0,52.0,43.0,Fog,93.0,100.0,63.0,0.00,30.23,30.30,30.19,52.0,63.0,42.0,4.0,10.0,0.0,0.0,3.0,14.0
2012-01-03,43.0,50.0,37.0,Fog,85.0,100.0,32.0,0.01,30.24,30.28,30.17,58.0,77.0,39.0,6.0,10.0,0.0,0.0,2.0,10.0
2012-01-04,42.0,47.0,37.0,,69.0,96.0,33.0,0.00,30.24,30.30,30.20,56.0,73.0,39.0,10.0,10.0,8.0,0.0,1.0,9.0
2012-01-05,42.0,51.0,36.0,,66.0,93.0,23.0,0.00,30.15,30.22,30.09,60.0,78.0,42.0,10.0,10.0,7.0,22.0,4.0,18.0


### Split Data

In [3]:
split_point = len(data) - 7
data_train, data_test = data[0:split_point], data[split_point:]

print('Training data: %d, Test data: %d' % (len(data_train), len(data_test)))

Training data: 1982, Test data: 7


In [4]:
def plot_helper(x, traces, trace_name, title):
    data = []
    
    for i, trace in enumerate(traces):
        # prepare data for plot
        data.append(
            Scatter(x=np.array(x),
                    y=trace,
                    name=trace_name[i])
        )

    layout = Layout({
        'title': title
        }
    )
    
    # plot scores over persistence values
    fig = Figure(data=data, layout=layout)
    py.iplot(fig)

### Develop Model

The data doens't have a strong seasonal component, but we decided to neutralize it and make it stationary by taking the seasonal difference. That is, we can take the observation for a day and subtract the observation from the same day one year ago.

We can invert this operation by adding the value of the observation one year ago. We will need to do this to any forecasts made by a model trained on the seasonally adjusted data.

In [5]:
# create a differenced series
def difference(data, interval=1):
    diff = list()
    for i in range(interval, len(data)):
        value = data[i] - data[i - interval]
        diff.append(value)
    return np.array(diff)

In [6]:
# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

In [7]:
# seasonal difference
X = data['temp_f_low'].values
days_in_year = 365
differenced = difference(X, days_in_year)

In [8]:
# fit model
model = ARIMA(differenced, order=(0, 1, 1))
model_fit = model.fit(disp=0)

# print summary of fit model
print(model_fit.summary())

                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                 1623
Model:                 ARIMA(0, 1, 1)   Log Likelihood               -5037.141
Method:                       css-mle   S.D. of innovations              5.390
Date:                Tue, 13 Jun 2017   AIC                          10080.282
Time:                        20:49:56   BIC                          10096.459
Sample:                             1   HQIC                         10086.285
                                                                              
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         -0.0009      0.086     -0.011      0.991        -0.170     0.168
ma.L1.D.y     -0.3572      0.030    -11.844      0.000        -0.416    -0.298
                                    Roots           

### One-Step Out-of-Sample Forecast
ARIMA models are great for one-step forecasts.

A one-step forecast is a forecast of the very next time step in the sequence from the available data used to fit the model.

The statsmodel ARIMAResults object provides a forecast() function for making predictions.

By default, this function makes a single step out-of-sample forecast. As such, we can call it directly and make our forecast. The result of the forecast() function is an array containing the forecast value, the standard error of the forecast, and the confidence interval information. Now, we are only interested in the first element of this forecast, as follows.

In [9]:
# one-step out-of sample forecast
forecast = model_fit.forecast()[0]

# invert the differenced forecast to something usable
forecast = inverse_difference(X, forecast, days_in_year)

print('Forecast: %f, Actual: %f' % (forecast, data_test.iloc[0]['temp_f_low']))

Forecast: 50.839022, Actual: 50.000000


### Multi-Step Out-of-Sample Forecast

We can also make multi-step forecasts using the forecast() function.

It is common with weather data to make one week (7-day) forecasts, so in this section we will look at predicting the minimum daily temperature for the next 7 out-of-sample time steps.

The forecast() function has an argument called steps that allows you to specify the number of time steps to forecast. By default, this argument is set to 1 for a one-step out-of-sample forecast. We can set it to 7 to get a forecast for the next 7 days.

In [10]:
# multi-step out-of-sample forecast
forecast = model_fit.forecast(steps=7)[0]

# invert the differenced forecast to something usable
history = [x for x in X]
day = 1

tests = []
predictions = []

for yhat in forecast:
    test = data_test.iloc[day - 1]['temp_f_low']
    predicted = inverse_difference(history, yhat, days_in_year)
    
    print('Day %d -- Forecast: %f, Actual: %f' % (day, predicted, test))
    history.append(predicted)
    day += 1
    
    tests.append(test)
    predictions.append(predicted)
    
test_score = np.sqrt(mean_squared_error(tests, predictions))
print('Test RMSE: %.3f' % test_score)

Day 1 -- Forecast: 50.839022, Actual: 50.000000
Day 2 -- Forecast: 48.838087, Actual: 52.000000
Day 3 -- Forecast: 43.837151, Actual: 53.000000
Day 4 -- Forecast: 41.836215, Actual: 51.000000
Day 5 -- Forecast: 41.835279, Actual: 55.000000
Day 6 -- Forecast: 45.834343, Actual: 48.000000
Day 7 -- Forecast: 50.833407, Actual: 48.000000
Test RMSE: 7.218


In [11]:
# plot predictions vs observations
plot_helper(data_test.index, [tests, predictions], ['Test', 'Forecast'], 'Line Plot of Predicted Values vs Test Dataset - San Luis Obispo, CA')